In [1]:
import sys
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
df = pd.read_csv('gk4z_very_clean.csv')
raw_text = ' '.join(df.clean_text.tolist() )
raw_text = raw_text.lower()
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)

In [3]:
seq_length = 5
dataX = []
dataY = []
for el in df.clean_text:
    raw_text_el = el.lower()
    n_chars_el = len(raw_text_el)
    for i in range(0, n_chars_el - seq_length, 1): # от нуля до длины текста без пяти
	      seq_in = raw_text_el[i:i + seq_length] # буквы в тексте от i до i+5 (5 букв)
	      seq_out = raw_text_el[i + seq_length] # буква в тексте под номером i+5
	      dataX.append([char_to_int[char] for char in seq_in]) # список из списков по 5 закодированных цифрами букв
	      dataY.append(char_to_int[seq_out]) # список букв, которые идут после каждой последовательности
n_patterns = len(dataX) # все 5-буквенные сочетания 
# print( "Total Patterns: ", n_patterns, len(dataY)) 

In [4]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
X = X / float(n_vocab)
y = np_utils.to_categorical(dataY)

In [5]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
filepath="weightsFIVEwithout_pudding-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [7]:
model.fit(X, y, epochs=10, batch_size=64, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
2604323/2604323 [==============================] - 1038s 398us/step - loss: 2.4952

Epoch 00001: loss improved from inf to 2.49519, saving model to weightsFIVEwithout_pudding-01-2.4952-bigger.hdf5
Epoch 2/10
2604323/2604323 [==============================] - 1037s 398us/step - loss: 2.1456

Epoch 00002: loss improved from 2.49519 to 2.14564, saving model to weightsFIVEwithout_pudding-02-2.1456-bigger.hdf5
Epoch 3/10
1072384/2604323 [===========>..................] - ETA: 10:14 - loss: 2.0504

KeyboardInterrupt: 

In [8]:
# save the model to file
model.save('model_gen2_FIVE_CHAR.h5')


In [9]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [17]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(50):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" у ког "
да на концерта после концерта после концерта после
Done.
